In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPool2D,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import pandas as pd
from tensorflow.keras.applications import MobileNet

In [2]:
model = MobileNet(input_shape=(224,224,3),include_top=False)

17235968/17225924 [==============================] - 3s 0us/step


In [3]:
train_datagen = ImageDataGenerator(rescale=1/255.,
                                  rotation_range=45,
                                  width_shift_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='reflect')

In [5]:
train_data = train_datagen.flow_from_directory('C:/Users/Venkatesh.DESKTOP-836KGPI/DSML_GL_PES/Deep Learning/Hackathon/train',
                                               target_size=(224,224),
                                               class_mode='binary',
                                               batch_size=32)

Found 1709 images belonging to 2 classes.


In [6]:
model.layers

In [13]:
model.layers[77]

In [7]:
model.trainable

True

In [8]:
for i in range(0,77):
  model.layers[i].trainable=False

In [9]:
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [10]:
base_model = Sequential()
base_model.add(model)
base_model.add(Flatten())
base_model.add(Dense(1024,activation='relu'))
base_model.add(Dense(512,activation='relu'))
base_model.add(Dropout(0.1))
base_model.add(Dense(64,activation='relu'))
base_model.add(Dense(1,activation='sigmoid'))

In [11]:
base_model.compile(optimizer=tf.keras.optimizers.SGD(0.01),loss='binary_crossentropy',metrics=['accuracy'])

In [12]:
base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 flatten (Flatten)           (None, 50176)             0         
                                                                 
 dense (Dense)               (None, 1024)              51381248  
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                32832     
                                                        

In [13]:
base_model.fit(train_data,epochs=20)

Epoch 1/20
54/54 [==============================] - 143s 3s/step - loss: 0.7781 - accuracy: 0.5688
Epoch 2/20
54/54 [==============================] - 133s 2s/step - loss: 0.6717 - accuracy: 0.5875
Epoch 3/20
54/54 [==============================] - 139s 3s/step - loss: 0.6338 - accuracy: 0.6530
Epoch 4/20
54/54 [==============================] - 130s 2s/step - loss: 0.6262 - accuracy: 0.6477
Epoch 5/20
54/54 [==============================] - 131s 2s/step - loss: 0.6017 - accuracy: 0.6706
Epoch 6/20
54/54 [==============================] - 130s 2s/step - loss: 0.5910 - accuracy: 0.6776
Epoch 7/20
54/54 [==============================] - 134s 2s/step - loss: 0.5847 - accuracy: 0.6788
Epoch 8/20
54/54 [==============================] - 136s 3s/step - loss: 0.5861 - accuracy: 0.6899
Epoch 9/20
54/54 [==============================] - 131s 2s/step - loss: 0.5550 - accuracy: 0.7185
Epoch 10/20
54/54 [==============================] - 131s 2s/step - loss: 0.5429 - accuracy: 0.7156
Epoch 11/

In [15]:
test = pd.read_csv('C:/Users/Venkatesh.DESKTOP-836KGPI/DSML_GL_PES/Deep Learning/Hackathon/test_(1).csv')
test.head()

,file_id
0,0
1,1
2,2
3,3
4,4


In [26]:
test.shape

(332, 1)

In [29]:
index=[]
pred=[]
for i in range(test.shape[0]):
  test_img = cv2.imread('C:/Users/Venkatesh.DESKTOP-836KGPI/DSML_GL_PES/Deep Learning/Hackathon/test/test/'+str(i)+'.jpg')
  test_img1 = cv2.resize(test_img,dsize=(224,224))
  test_img2 = test_img1/255.
  test_img3 = np.expand_dims(test_img2,0)
  pred.append(round(base_model.predict(test_img3)[0][0]))
  index.append(i)

In [30]:
pred_df = pd.DataFrame({'file_id':index,'label':pred})
pred_df.to_csv('SushmaD.csv')